## Reference Counting

Method that returns the reference count for a given variable's memory address:

In [1]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

make a variable, and check it's reference count:

In [2]:
my_var = [1, 2, 3, 4]
ref_count(id(my_var))

1

There is another built-in function we can use to obtain the reference count:

In [3]:
import sys
sys.getrefcount(my_var)

2

But why is this returning 2, instead of the expected 1 we obtained with the previous function?

Answer: The *sys.getrefcount()* function takes **my_var** as an argument, this means it receives (and stores) a reference to **my_var**'s memory address **also** - hence the count is off by 1. So we will use *from_address()* instead.

We make another reference to the **same** reference as `my_var`:

In [4]:
other_var = my_var

Let's look at the memory address of those two variables and the reference counts:

In [5]:
print(hex(id(my_var)), hex(id(other_var)))
print(ref_count(id(my_var)))

0x1e43f368388 0x1e43f368388
2


Force one reference to go away:

In [6]:
other_var = None

And we look at the reference count again:

In [7]:
print(ref_count(id(my_var)))

1


We see that the reference count has gone back to 1.

## Garbage Collection

In [3]:
import ctypes
import gc

In [4]:
def ref_count(address):
    return ctypes.c_long.from_address(address).value


In [5]:
def object_by_id(object_id):
    for obj in gc.get_objects():
        if id(obj) == object_id:
            return "Object exists"
    return "Not found"

In [6]:
class A:
    def __init__(self):
        self.b = B(self)
        print('A: self: {0}, b:{1}'.format(hex(id(self)), hex(id(self.b))))

In [7]:
class B:
    def __init__(self, a):
        self.a = a
        print('B: self: {0}, a: {1}'.format(hex(id(self)), hex(id(self.a))))

We turn off the GC so we can see how reference counts are affected when the GC does not run and when it does (by running it manually).

In [8]:
gc.disable()

Now we create an instance of A, which will, in turn, create an instance of B which will store a reference to the calling A instance

In [10]:
my_var = A()

B: self: 0x7f91fe44c7f0, a: 0x7f91fe44c4f0
A: self: 0x7f91fe44c4f0, b:0x7f91fe44c7f0


As we can see A and B's constructors ran, and we also see from the memory addresses that we have a circular reference.

In fact `my_var` is also a reference to the same A instance:

In [12]:
print(hex(id(my_var)))

0x7f91fe44c4f0


Another way to see this:

In [14]:
print('a: \t{0}'.format(hex(id(my_var))))
print('a.b: \t{0}'.format(hex(id(my_var.b))))
print('b.a: \t{0}'.format(hex(id(my_var.b.a))))

a: 	0x7f91fe44c4f0
a.b: 	0x7f91fe44c7f0
b.a: 	0x7f91fe44c4f0


In [15]:
a_id = id(my_var)
b_id = id(my_var.b)

We can see how many references we have for `a` and `b`: